In [143]:
from splinter import Browser
from bs4 import BeautifulSoup as soup
from webdriver_manager.chrome import ChromeDriverManager
import re

# Set up Splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

# Visit the Mars Facts site
url = 'https://www.goodreads.com/list/show/1.Best_Books_Ever'
browser.visit(url)

html = browser.html
html_soup = soup(html, 'html.parser')

In [148]:
def GoodScrapes(pages, count=1, best_books={}):
    while count < pages:
        html = browser.html
        html_soup = soup(html, 'html.parser')
        close_popup = 'div[class="modal__close"]'
        try:
            browser.find_by_tag(close_popup).button.click()
        except:
            pass
        table = html_soup.find('table', class_='tableList')
        rows = table.find_all('tr')
        for row in rows:
            cols = row.find_all('td')
            num = int(cols[0].text)
            info = cols[2]
            name = info.a.text.strip()
            auth = info.findNext(class_='authorName').text
            stars = info.findNext(class_='minirating').text
            rating = re.findall(r'\d+\.\d+', stars)[0]
            best_books[num] = [name, auth, float(rating)]
        tg = 'a[class="next_page"]'
        try:
            browser.find_by_tag(tg).click()
        except:
            browser.find_by_tag(close_popup).button.click()
            browser.find_by_tag(tg).click()
        count += 1
    return best_books

In [ ]:
# before running the next cells, sign in to goodreads so their stupid popup doesnt ruin the scraping

In [149]:
url = 'https://www.goodreads.com/list/show/1.Best_Books_Ever'
browser.visit(url)

In [150]:
best_books = GoodScrapes(20)

In [151]:
best_books

{1: ['The Hunger Games (The Hunger Games, #1)', 'Suzanne Collins', 4.33],
 2: ['Harry Potter and the Order of the Phoenix (Harry Potter, #5)',
  'J.K. Rowling',
  4.5],
 3: ['Pride and Prejudice', 'Jane Austen', 4.28],
 4: ['To Kill a Mockingbird', 'Harper Lee', 4.27],
 5: ['The Book Thief', 'Markus Zusak', 4.39],
 6: ['Twilight (The Twilight Saga, #1)', 'Stephenie Meyer', 3.64],
 7: ['Animal Farm', 'George Orwell', 3.98],
 8: ['The Chronicles of Narnia (Chronicles of Narnia, #1-7)',
  'C. S. Lewis',
  4.27],
 9: ['J.R.R. Tolkien 4-Book Boxed Set: The Hobbit and The Lord of the Rings',
  'J.R.R. Tolkien',
  4.61],
 10: ['The Fault in Our Stars', 'John Green', 4.15],
 11: ['Gone with the Wind', 'Margaret Mitchell', 4.3],
 12: ['The Giving Tree', 'Shel Silverstein', 4.38],
 13: ['Wuthering Heights', 'Emily Brontë', 3.88],
 14: ['The Picture of Dorian Gray', 'Oscar Wilde', 4.12],
 15: ['Jane Eyre', 'Charlotte Brontë', 4.14],
 16: ['The Perks of Being a Wallflower', 'Stephen Chbosky', 4.22

In [152]:
browser.quit()

In [153]:
import pandas as pd

In [156]:
books_df = pd.DataFrame.from_dict(best_books, orient='index', columns=['name', 'author', 'stars'])
books_df.head()

,name,author,stars
1,"The Hunger Games (The Hunger Games, #1)",Suzanne Collins,4.33
2,Harry Potter and the Order of the Phoenix (Har...,J.K. Rowling,4.50
3,Pride and Prejudice,Jane Austen,4.28
4,To Kill a Mockingbird,Harper Lee,4.27
5,The Book Thief,Markus Zusak,4.39


In [157]:
books_df.to_csv('goodreads_best_books.csv')